# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [2]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 550b60d3


In [3]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [24]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [25]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [8]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [26]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/stevegoodman/dev/AIE9/07_Deep_Agents/workspace


In [27]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] comprehensive_morning_routine_guide.md (12729 bytes)
[FILE] morning_routine_guide.md (31474 bytes)
[DIR] research
[FILE] sleep_improvement_plan.md (5790 bytes)
[FILE] stress_management_comprehensive_guide.md (21293 bytes)
[FILE] stress_management_findings.md (2761 bytes)
[FILE] stress_management_findings.txt (3323 bytes)
[FILE] stress_management_research_report.md (16295 bytes)


In [28]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [29]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [30]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/stevegoodman/dev/AIE9/07_Deep_Agents/workspace


In [19]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Sleep Improvement Plan is Ready! 🌙

I've created a comprehensive 4-week sleep improvement plan specifically tailored to address your three main issues: inconsistent bedtime, phone use in bed, and morning fatigue. The plan is saved as `/sleep_improvement_plan.md` for easy reference.

### Key Highlights of Your Plan:

**Immediate Actions (Start Tonight)**:
- Set a consistent bedtime of 10:30pm (middle of your current range)
- Move your phone charger outside the bedroom
- Create a 1-hour wind-down routine starting at 9:30pm

**Week-by-Week Progression**:
1. **Week 1**: Foundation building with consistent routine
2. **Week 2**: Habit reinforcement and environment optimization  
3. **Week 3**: Advanced strategies and fine-tuning
4. **Week 4**: Long-term sustainability planning

**Specific Solutions**:
- **Inconsistent bedtime**: Gradual schedule adjustment with ±30-minute flexibility
- **Phone in bed**: Complete bedroom phone ban with engaging alternativ

In [20]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_9] Provide implementation guidance (completed)


Workspace contents:
  [DIR] research/
  [FILE] sleep_improvement_plan.md (5790 bytes)


---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
First the definition of "complex task" might be hard to identify and some tasks might have the ablitiy be completed in 1 shot.
 Second, how many tasks is the sweet spot to be just detailed enough, but not a absurdly large number of minor tasks and not so few as to be near-indistiguishable from a shallow agent that completes a job in 1 or 2 steps.

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
Offload tool_call results to trim the contenxt.
Metrics should be persisted to long-term memory to be saved and then retreived at  a later date
User conditions should always remain in context. It may be wise to re-inforce their importance by, for example, calling them out at near the beginning AND near the end of the prompt so that they are not "lost in the middle" of context and potentially ignored. May also have a reflection node that checks any responses don't violate any of the known user conditions.



---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [64]:
# Allowlist of external directories
ALLOWED_READS = [
    Path(workspace_path.parent/"data")
]

def is_allowed(path: Path) -> bool:
    path = path.resolve()
    return any(str(path).startswith(str(root)) for root in ALLOWED_READS)

@tool
def read_external_file(path: str) -> str:
    """
    Read a readonly file from an allowed external directory.
    """
    file_path = Path(workspace_path.parent/path)

    if not is_allowed(file_path):
        raise PermissionError(f"Access denied: {file_path}")

    if not file_path.exists() or not file_path.is_file():
        raise FileNotFoundError(f"File not found: {file_path}")

    return file_path.read_text(encoding="utf-8")

In [65]:
### YOUR CODE HERE ###

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide


# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")


TODO_STORE.clear()

file_system =  FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True
)



# Step 3: Create the agent with a research-focused system prompt

researcher_prompt = """ Your role is a world class wellness researcher.
Create a research report based on the user's query.
You have access to the following custom tools
ls, edit_file, read_file, write_file
You have access to the document data/HealthWellnessGuide.txt accsessible via read_external_file_tool as the primary source of your research. 
You should read from this file but NEVER edit or write to it.


Break down the problem into small TODO steps. 

1. First, create a todo list to track your progress. Number each step in ascending numeric order
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a structred markdown report when complete. The format should contain following headings - Introduction, Review, Conclusions/Recommendations, References. 

Be clear and concise. 

"""


agent = create_deep_agent(
    model = init_chat_model('claude-sonnet-4-5-20250929'),
    backend = file_system,
    tools = [read_external_file],
    system_prompt=researcher_prompt
)

# Step 4: Test with the stress management research task

# agent.invoke("messages: [HumanMessage(content="Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies.")] )

In [68]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage
agent.invoke({"messages": [HumanMessage(content="Read in the healthwellnessguide document and tell me when you've done it")]})

{'messages': [HumanMessage(content="Read in the healthwellnessguide document and tell me when you've done it", additional_kwargs={}, response_metadata={}, id='f9b12146-7dd6-46c2-b8f0-989f6a45b9aa'),
  AIMessage(content=[{'text': "I'll read the HealthWellnessGuide document for you now.", 'type': 'text'}, {'id': 'toolu_011bsmamsjBh8oyrcToKraDQ', 'input': {'path': 'data/HealthWellnessGuide.txt'}, 'name': 'read_external_file', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01HaJBrjSx6LURd4tpBizsk7', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 6221}, 'cache_creation_input_tokens': 6221, 'cache_read_input_tokens': 0, 'input_tokens': 3, 'output_tokens': 84, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'anthropic'}, id='lc_run--019c2e63-2dfd-7db1-bebe-b75f572ad17d-

In [70]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage
agent.invoke({"messages": [HumanMessage(content="Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies.")] })

{'messages': [HumanMessage(content='Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies.', additional_kwargs={}, response_metadata={}, id='9642f3ec-9e9c-4fe0-8ed0-cfa3cac62aff'),
  AIMessage(content=[{'text': "I'll help you create a comprehensive research report on stress management techniques. Let me start by organizing this work into clear steps.", 'type': 'text'}, {'id': 'toolu_011e56SrMiKjDq4PWKkUu9Tf', 'input': {'todos': [{'content': 'Read the primary source document (HealthWellnessGuide.txt) to gather information on stress management techniques', 'status': 'in_progress'}, {'content': 'Identify and extract at least 5 evidence-based stress management strategies from the source', 'status': 'pending'}, {'content': 'Organize findings and create structured notes on each strategy', 'status': 'pending'}, {'content': 'Write a comprehensive research report with Introduction, Review, Conclusions/Recommendations, and References sec

In [ ]:
## `internet_search` to be added as a tool to compare output
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

agent = create_deep_agent(
    tools=[internet_search]
)

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [71]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [72]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


{}

In [74]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what was accomplished:

## ✅ **Project Complete - All Tasks Finished**

### **🔬 Scientific Research Foundation**
The research-agent conducted thorough scientific research covering:
- Circadian rhythm science and light exposure effects
- Hormonal responses (cortisol, melatonin, endorphins) to morning activities
- Evidence on morning exercise and sustained energy levels
- Studies on nutrition's impact on daily energy
- Research on mindfulness practices and cognitive performance
- Sleep-wake cycle optimization strategies

### **📝 Comprehensive Guide Creation**
The writing-agent created a well-structured, practical guide featuring:
- **Science-backed foundation** with research citations
- **4 core components**: Sleep optimization, exercise, nutrition, and mindset
- **3 flexible routines**: 15-minute (busy), 30-minute (moderate), 60-minute (comprehensive)
- **Implement

In [75]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save guide as formatted markdown file (completed)

Generated files in workspace:
  [FILE] comprehensive_morning_routine_guide.md (12729 bytes)
  [FILE] morning_routine_guide.md (31474 bytes)
  [DIR] research/
  [FILE] sleep_improvement_plan.md (5790 bytes)
  [FILE] stress_management_comprehensive_guide.md (21293 bytes)
  [FILE] stress_management_findings.md (2761 bytes)
  [FILE] stress_management_findings.txt (3323 bytes)
  [FILE] stress_management_research_report.md (16295 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [78]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [79]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [80]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [81]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! Great to see you again. Based on your profile, I can see your primary goal is to improve energy levels and get better sleep, and you prefer morning workouts. I also note you have mild anxiety, which I'll keep in mind for my recommendations.

Here's a personalized exercise routine that aligns perfectly with your goals:

## **Morning Energy & Sleep Enhancement Routine**

### **Week Schedule (4-5 days/week):**

**Monday, Wednesday, Friday - Energizing Cardio + Strength**
- **5-minute dynamic warm-up** (arm circles, leg swings, gentle stretching)
- **20-25 minutes moderate cardio**: 
  - Brisk walking/light jogging
  - Cycling 
  - Swimming (excellent for anxiety relief)
- **15-20 minutes strength training**:
  - Bodyweight exercises: push-ups, squats, lunges, planks
  - Light weights if available
- **5-minute cool-down with deep breathing**

**Tuesday, Thursday - Gentle Movement & Flexibility**
- **15-20 minutes yoga flow** (particularly beneficial for anxiety)
- 

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [38]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [28]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [29]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [30]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [31]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Summary

Based on your profile as a 35-year-old sedentary office worker looking to lose 15 pounds while dealing with sleep issues, I've created a comprehensive wellness assessment. Here are the key findings and recommendations:

### **Top Priority: Sleep Optimization** 
Your sleep issues are the foundation that needs addressing first - poor sleep disrupts weight loss hormones and energy levels needed for other changes.

### **Your Action Plan:**

**🟢 Start Today:**
1. **Sleep hygiene setup** - consistent bedtime, no screens 1 hour before bed
2. **Hourly movement breaks** - stand and move for 2-3 minutes every hour
3. **Hydration baseline** - water upon waking, keep bottle at desk

**🟡 Next 1-2 Weeks:**
1. **Daily 20-30 minute walks** plus 2-3 bodyweight strength sessions
2. **Consistent sleep schedule** (same bedtime/wake time daily)
3. **Food logging** for one week to identify patterns

**🔴 1-3 Month Goals:**
1. **150+ minutes cardio weekly*

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [32]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli not installed. Install with:
  uv pip install deepagents-cli
  # or
  pip install deepagents-cli


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [33]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [34]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [35]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! Alex, your comprehensive 2-week wellness program is complete and organized! 🎉

## What I've Created For You:

### **Main Program File:**
- **Alex_Complete_2Week_Wellness_Program.md** - Your master guide with daily integration schedules, weekly progression, and success tracking

### **Individual Reference Files:**
- **Alex_Exercise_Plan.md** - Complete 3x/week workout routines with progressions
- **Alex_Nutrition_Plan.md** - 14 days of vegetarian meals with shopping lists and prep tips  
- **Alex_Mindfulness_Plan.md** - Daily stress management and sleep optimization techniques

### **Additional Resources:**
- Shopping lists for both weeks
- Meal prep guidelines
- Progress tracking suggestions

## Key Program Features:

✅ **Tailored to Your Preferences**: Morning exercise, vegetarian diet, detailed guidance
✅ **Addresses Your Goals**: Consistent routine, energy improvement, stress/sleep management  
✅ **Considers Your Conditions**: Anxiety-friendly techn

In [36]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Assess user profile and preferences (completed)
✅ [todo_3] Create exercise plan (completed)
✅ [todo_5] Create nutrition plan (completed)
✅ [todo_7] Create stress & sleep optimization plan (completed)
✅ [todo_9] Integrate all plans into cohesive program (completed)
✅ [todo_11] Save individual plans to files (completed)
✅ [todo_13] Save complete program overview (completed)

GENERATED FILES
  [FILE] .gitkeep (0 bytes)
  [FILE] Alex_2_Week_Exercise_Program.txt (4330 bytes)
  [FILE] Alex_Complete_2Week_Wellness_Program.md (4757 bytes)
  [FILE] Alex_Exercise_Plan.md (3864 bytes)
  [FILE] Alex_Mindfulness_Plan.md (6396 bytes)
  [FILE] Alex_Nutrition_Plan.md (5482 bytes)
  [FILE] complete_14_day_vegetarian_meal_plan.txt (5045 bytes)
  [FILE] comprehensive_morning_routine_guide.md (25117 bytes)
  [FILE] meal_plan_nutritional_highlights.txt (468 bytes)
  [FILE] meal_plan_week_1.txt (1765 bytes)
  [FILE] meal_plan_week_2.txt (1598 bytes)
  [FILE] meal_prep_tips.txt (

In [37]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of morning_routine_guide.md:
# The Ultimate Morning Routine Guide: Energize Your Day for Peak Performance

## Table of Contents
- [Introduction: Why Your Morning Sets the Tone](#introduction-why-your-morning-sets-the-tone)
- [The Science-Backed Foundation](#the-science-backed-foundation)
- [Exercise: Wake Up Your Body](#exercise-wake-up-your-body)
- [Nutrition: Fuel Your Success](#nutrition-fuel-your-success)
- [Mindset Practices: Train Your Mental Muscle](#mindset-practices-train-your-mental-muscle)
- [Implementation Strategies: Building Lasting Habits](#implementation-strategies-building-lasting-habits)
- [Common Mistakes to Avoid](#common-mistakes-to-avoid)
- [Customization for Your Lifestyle](#customization-for-your-lifestyle)
- [Sample Morning Routine Templates](#sample-morning-routine-templates)
- [Your Next Steps](#your-next-steps)

---

## Introduction: Why Your Morning Sets the Tone

Picture this: You wake up feeling refreshed, energized, and ready to tackle whatever

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
*Your answer here*

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:

Persistant storage
RBAC controls over the file system for read/write/edits
Subagent management - e.g. using cheaper, faster LLMs to perform isolated tasks 

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [ ]:
#  I'm not building a wellness tracker. 
# Continuing on From Homework 2 , I'm developing parts of my proposed Certfication Challenge project

# PRoblem Statement:
# Simply put, its a bank loans application automated review process for banks that lend to companies (as opposed to consumers).
# To turn a process that currently (and very manually) takes bank employees > 1hr per application. Cut employee time to around c. 5 minutes


# There are a few elements to this that the agent has to research : (this  is only a subset for the purposes of this assignment)

# 1. Examine the contents of the Loan application, and the kind of loan being applied for
# 2. Check Bank Loan Policy documents, check meet basic eligibility criteria for loan and what  appropriate documentation should be provided by customer.
# 3. Validate the presence and contents of said documents
# 4. Lookup company information in Public Register of Companies (Companies House in UK), look at the company directors 
# 5. Do internet searches for online presence of company, website, linked in , news. 

# Then evaluate whether

# 1. company is legit, is a going concern, and has the finances and meets the basic loan eligibilty citeria
# 2. information is consistant across data sources
# 3. flag any signficant concerns
# 4. Revert back to the customer to request missing information (HITL).
# 5. On receipt of a response, update the process with the new information, and resume the process. This may take days to weeks, so
# and may require a completly new submission, so persist to LT storage.


# Design is therefore:

# 1. Planning - what do we need to research - depends on loan application charateristics (not all agents may be required)
# 2. Context management - load some of the loan policy docs required, compress tool_conversations  
# 3. Tools - Websearch, LinkedIn search (via MCP server), Companies House Search (via MCP server),
# 4. Specialist Sub - Agents - Social Media, Companies House, Internet Searches
# 5. LT Memory - user attributes in this context are : XXXXXX
# 6. User Input (first message) - loan application details 
# 7 HITL - early in process after first message to verify company on application is the same company 
#  found in Linkedn, Companies House etc (since name clashes, trading-as names may cause confusion)
# 8 HITL to ask for any followup information
# 9 GAURDRAILS - verification that our assessment is correct. PII redaction fo   My output details for

### Step 1: Define Tools

In [52]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


In [120]:
output_dir = WORKSPACE.parent / "loan_policy_documents"

output_dir

PosixPath('loan_policy_documents')

In [121]:
# Bank Policy Docs

# Prob can do this in langchain 

#from rich.markdown import Markdown
from docling.document_converter import DocumentConverter
#from docling.chunking import HybridChunker

#from docling_core.transforms.chunker.tokenizer.openai import OpenAITokenizer


converter = DocumentConverter()

KB_URL = "https://www.fundingoptions.com/knowledge/"
sources = [
    "asset-finance", "bridging-loans", "invoice-finance", "merchant-cash-advance", "revolving-credit-facility", "property-development-finance",
    "business-loans", "working-capital-finance", "short-term-business-loans", "invoice-discounting", "invoice-factoring-vs-invoice-discounting",
    "loans-for-limited-company", "compare-sole-trader-loans-with-funding-options-by-tide", "finance-leases", "debentures-floating-charges",
    "trade-finance", "how-to-apply-for-the-growth-guarantee-scheme", "invoice-factoring", "debt-financing", "commercial-property-finance",
    "find-a-semi-commercial-mortgage-lender", "business-loans-for-women", "unsecured-business-loans", "secured-business-loans", 
    "business-loans-for-bad-credit", "alternative-finance", "auction-finance", "construction-finance", "equity-finance"
    ]

# Scrape the webpages and create the corpus as both text and markdown
# markdown might provide some extra structure for finding good chunks

corpus_plain_txt = []

output_dir = WORKSPACE.parent / "loan_policy_documents"
corpus_docs = []  # Store the actual DoclingDocument objects
for source in sources:
    doc = converter.convert(KB_URL + source).document
    # Export Markdown format:
    with (output_dir / f"{source}.md").open("w", encoding="utf-8") as fp:
        fp.write(doc.export_to_markdown())

    #to_do : write the doc to the loan_policy_docs folder



In [122]:
file_system =  FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True
)

# Allowlist of external directories
ALLOWED_READS = [
    Path(workspace_path.parent/"loan_policy_documents")
]

def is_allowed(path: Path) -> bool:
    path = path.resolve()
    return any(str(path).startswith(str(root)) for root in ALLOWED_READS)

@tool
def read_external_file(path: str) -> str:
    """
    Read a readonly file from an allowed external directory.
    """
    file_path = Path(workspace_path.parent/path)

    if not is_allowed(file_path):
        raise PermissionError(f"Access denied: {file_path}")

    if not file_path.exists() or not file_path.is_file():
        raise FileNotFoundError(f"File not found: {file_path}")

    return file_path.read_text(encoding="utf-8")

In [ ]:
# Test 

In [ ]:
# question is how we utilise this information - could be a skill or md files loaded like we did with the first example.
# All w e need is the correct loan type from the application to find the correct file.




In [ ]:
# MCP Tools setup

# Linked In Server and Companies House MCP Services
# running these locally for now, porting them to GCP Cloud Run for Cert Challenge.


from pathlib import Path
from langchain_mcp_adapters.client import MultiServerMCPClient
import shutil

# Run MCP server from project root so it can start correctly (avoids "Connection closed")
project_root = Path.cwd()
try:
    COMPANIES_HOUSE_API_KEY =  os.environ["COMPANIES_HOUSE_API_KEY"]
except KeyError:
    raise ValueError("COMPANIES_HOUSE_API_KEY is not set. Run the cell above and ensure .env in project root contains it.")

# Find npx path - subprocess may not have same PATH as notebook
npx_path = shutil.which("npx") or "/opt/homebrew/bin/npx"

# Ensure PATH includes homebrew bin for subprocess
env = os.environ.copy()
if "/opt/homebrew/bin" not in env.get("PATH", ""):
    env["PATH"] = f"/opt/homebrew/bin:{env.get('PATH', '')}"

ch_client = MultiServerMCPClient(
    {
        "companies-house": {
            "command": npx_path,  # Use full path to npx
            "transport": "stdio",
            "args": ["-y", "companies-house-mcp-server"],
            "env": {
                **env,  # Include PATH in environment
                "COMPANIES_HOUSE_API_KEY": (COMPANIES_HOUSE_API_KEY or "")
            }
        }
    }
)
ch_tools = await ch_client.get_tools()

In [102]:
ch_tools

[StructuredTool(name='search_companies', description='Search for companies by name or company number', args_schema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Company name or number to search for'}, 'items_per_page': {'type': 'number', 'description': 'Number of results per page (1-100)', 'default': 20}, 'start_index': {'type': 'number', 'description': 'Starting index for pagination', 'default': 0}}, 'required': ['query']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1036dae80>),
 StructuredTool(name='get_company_profile', description='Get detailed profile information for a specific company', args_schema={'type': 'object', 'properties': {'company_number': {'type': 'string', 'description': 'The company number'}}, 'required': ['company_number']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1183e0d60>),
 Stru

In [58]:
# Test. 
from langchain.agents import create_agent

agent = create_agent(
    "gpt-4o",
    tools = ch_tools  
)
ch_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "get details for Goodmans Consulting"}]}
)

In [59]:
ch_response

{'messages': [HumanMessage(content='get details for Goodmans Consulting', additional_kwargs={}, response_metadata={}, id='9be4febb-4028-46ad-8a46-0b43698c7f62'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 1962, 'total_tokens': 1979, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ad98c18a04', 'id': 'chatcmpl-D6exO8x6ELoQvK3gIwp0orM6lxA6H', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c38d1-75ff-76e0-a032-d6d5e3355cb0-0', tool_calls=[{'name': 'search_companies', 'args': {'query': 'Goodmans Consulting'}, 'id': 'call_Bs45THmGFWZqAaYe8tXTOqIP', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'i

In [41]:
# Connect to LinkedIn MCP
# Run the MCP server from its own directory so uv finds the project (avoids "Connection closed").
# Prerequisites (from linkedin-mcp-server dir):
#   uv run linkedin-mcp-server --get-session   # create LinkedIn session once
#   uv run playwright install chromium         # if you see "Failed to start browser"

uv_path = shutil.which("uv") or "/opt/homebrew/bin/uv"
linkedin_server_dir = os.path.expanduser("~/dev/linkedin-mcp-server")
env = os.environ.copy()
if "/opt/homebrew/bin" not in env.get("PATH", ""):
    env["PATH"] = f"/opt/homebrew/bin:{env.get('PATH', '')}"
env["TRANSPORT"] = "stdio"

li_client = MultiServerMCPClient(
    {
        "linkedin": {
            "command": uv_path,
            "transport": "stdio",
            "args": ["--directory", linkedin_server_dir, "run", "linkedin-mcp-server"],
            "cwd": linkedin_server_dir,
            "env": env,
        }
    }
)
li_tools = await li_client.get_tools()

In [42]:
li_tools

[StructuredTool(name='get_person_profile', description='Get a specific person\'s LinkedIn profile.\n\nArgs:\n    linkedin_username: LinkedIn username (e.g., "stickerdaniel", "williamhgates")\n    ctx: FastMCP context for progress reporting\n\nReturns:\n    Structured data from the person\'s profile including:\n    - linkedin_url, name, location, about, open_to_work\n    - experiences: List of work history (position_title, institution_name,\n      linkedin_url, from_date, to_date, duration, location, description)\n    - educations: List of education (institution_name, degree, linkedin_url,\n      from_date, to_date, description)\n    - interests: List of interests with category (company, group, school,\n      newsletter, influencer) and linkedin_url\n    - accomplishments: List of accomplishments (category, title)\n    - contacts: List of contact info (type: email/phone/website/linkedin/\n      twitter/birthday/address, value, label)', args_schema={'properties': {'linkedin_username': {'

In [88]:
# internet search

from tavily import TavilyClient
tavily_client = TavilyClient()

@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

In [ ]:
# Test

### Step 2: Define your subagent configurations


In [64]:
# PROMPTS for LI CH Websearch

CH_PROMPT = """
<Role>You are a financial investigator tasked with checking the status of  registered companies. Assume todays date is 15 May 2025 </Role>

<Task>

Check details of a company (name provided ) the Companies House (UK) database. 

This is the central database of all registered companies in the United Kingdom.  If the company is a private or public limited company, they should appear in this database.

Check that the company name provided exists on the database. If an exact match cannot be found request more details
from the user and redo the search again. If there are multiple matches then ask the user to identify the correct one 
from a list.


Things to pay careful attention to:
-Office Address - the address on application form  may not match the address in Companys House. This is ok, but should be flagged.
-Company Status - tells whether a going concern
-Nature of Business - what products and services they provide
-Incorporation date and desolution dates
- People - names of officers  or persons with signficant control, their position within the company, And the percentages of shares they may own.
- there are no issues regarding annual submissions  being overdue.  

If Company is not a going concern - e.g. is insolvant, or is behind in its findings, then these should be flagged.

Consistancy checks between the information provided in the context and the information from Commpanies House should be done for:
-Length of time in business
-Status of business is a going concern and not dissolved or bankrupt
- If the applicant details or directors with signficant influence are provided, then check these are consistant with company posts, directors or positions
in the database. 


Do not make any general opinions or recommentations. Only share your findings.
</Task>

<User details>
{company_context}
</User details>

"""

In [ ]:
ch_sub_agent = create_deep_agent(
    model = init_chat_model('claude-sonnet-4-5-20250929'),
    backend = file_system,
    tools = [read_external_file] + ch_tools,
    system_prompt=CH_PROMPT
)

results = await ch_sub_agent.ainvoke(
    {
        "messages": [HumanMessage(content=application1_str)]
    }
)


In [68]:
print(results["messages"][-1].content)

Now I have the complete picture. Let me compile my findings:

---

## COMPANIES HOUSE VERIFICATION REPORT - GOODMANS CONSULTING

**Date of Report:** 15 May 2025  
**Applicant:** Steven Goodman  
**Company Name Provided:** Goodmans Consulting

---

### CRITICAL FINDINGS

#### 1. **COMPANY NOT FOUND AS ACTIVE ENTITY**

No active company with the name "Goodmans Consulting" or similar variations exists on the Companies House database.

#### 2. **DISSOLVED COMPANY IDENTIFIED**

A company named **"GOODMAN'S CONSULTING LIMITED"** (Company Number: **08139267**) was found, but:

- **Company Status:** DISSOLVED
- **Date of Incorporation:** 11 July 2012
- **Date of Dissolution:** 28 December 2017
- **Registered Office Address:** Heskin Hall Farm Wood Lane, Heskin, Preston, PR7 5PA

#### 3. **APPLICANT CONNECTION CONFIRMED**

Steven Goodman (born October 1972, residing at 3 Manor Road, Ruislip, HA4 7LA) was confirmed as a director of the dissolved company:
- **Appointment Date:** 13 July 2012
- **

In [74]:
LI_PROMPT = """
<Role> Your role is a financial investigator </Role>
<Task> Search for a company via Linked-in professional networking website.
Search if the company has a linked in page. Scape any information that links it to a company webite
or HQ or office address.
Look for any personal profiles that are connected to the company, particularly if they list themselves
as directors or other positions of signficiant control or influence over the company

Things to pay careful attention to:
--Nature of business products and services
--Signficant people associated with the business
</Task>

<User details>
{company_context}
</User details>
"""

In [87]:
shortened_str = """
Applicant Name: Steven Goodman
Company Name: Goodmans Consulting
Address: Ruislip, UK
What is your industry?: Business Support Services
When did you start trading?: 11/07/2012
"""

In [85]:
#Note put some limits on the calls - 

li_sub_agent = create_deep_agent(
    model = init_chat_model('claude-sonnet-4-5-20250929'),
    backend = file_system,
    tools = [read_external_file] + li_tools,
    system_prompt=LI_PROMPT
)

results = await li_sub_agent.ainvoke(
    {
        "messages": [HumanMessage(content=shortened_str)]
    }
)


In [86]:
print(results['messages'][-1].content)

## FINANCIAL INVESTIGATION REPORT
**Subject:** Goodmans Consulting  
**Applicant:** Steven Goodman  
**Location:** Ruislip, UK  
**Date:** Investigation Conducted via LinkedIn

---

## EXECUTIVE SUMMARY

This investigation reveals **LIMITED VERIFIABLE INFORMATION** about Goodmans Consulting and Steven Goodman via LinkedIn. While the company has a minimal LinkedIn presence, critical business details are absent, and no personal LinkedIn profile could be definitively linked to Steven Goodman of Goodmans Consulting in Ruislip.

**⚠️ RED FLAGS:**
- Minimal digital footprint for a consulting business
- No identifiable directors or key personnel on LinkedIn
- No company website, contact details, or business information available
- No employee listings or company posts
- Unable to verify Steven Goodman's connection to the business

---

## DETAILED FINDINGS

### 1. COMPANY PROFILE: GOODMANS CONSULTING

**LinkedIn Presence:** ✓ EXISTS (but minimal)  
**Company Page:** https://www.linkedin.com/c

In [ ]:


INTERNET_SEARCH_PROMPT = """
Search the internet for the company name provided in the context.
Note that the name provided may not be an exact match for the company found on the internet. Multiple companies
may match this company name, or the company may maintain a separate brand or trading-as name and a registered company name.
Other companies may also match. 
Look for a company website and verify if the contents of the webside match the details provided in the context.

DONT look at linkedin - this is already being taken care of.

So use any other information provided to help narrow the search:
    Location of office is a match
    Directors or employees of the company match.
    Type or nature of business is a match.

Look for news on the company and summarise the content of the news story, indicating anything that may be negative 
or related to their financial or trading position

Include citations and links on any sources used in your assessment. 

<User details>
{company_context}
</User details>
"""

In [91]:
internet_search_sub_agent = create_deep_agent(
    model = init_chat_model('claude-sonnet-4-5-20250929'),
    backend = file_system,
    tools = [read_external_file, internet_search],
    system_prompt=INTERNET_SEARCH_PROMPT
)

results = await li_sub_agent.ainvoke(
    {
        "messages": [HumanMessage(content=shortened_str)]
    }
)


In [95]:
print(results["messages"][-1].content)

## **FINANCIAL INVESTIGATION REPORT: GOODMANS CONSULTING**

### **Company Details Provided:**
- **Applicant Name:** Steven Goodman
- **Company Name:** Goodmans Consulting
- **Address:** Ruislip, UK
- **Industry:** Business Support Services
- **Trading Start Date:** 11/07/2012

---

## **LINKEDIN INVESTIGATION FINDINGS:**

### **Company LinkedIn Presence:**

I searched multiple variations of the company name on LinkedIn:

1. **"goodmans-consulting"** - Found a company page but with **NO detailed information**:
   - LinkedIn URL exists: https://www.linkedin.com/company/goodmans-consulting/
   - No "About Us" section
   - No website listed
   - No phone number
   - No headquarters information
   - No employee listings visible
   - **No posts or activity** on the company page

2. **"goodman-consulting"** (singular) - Found another page:
   - LinkedIn URL: https://www.linkedin.com/company/goodman-consulting/
   - Industry: Business Consulting and Services
   - Company Size: 2-10 employees
 

In [108]:
internet_subagent = {
    "name": "internet-search-agent",
    "description": "Use this agent to look for a company website or news about the company.",
    "system_prompt": INTERNET_SEARCH_PROMPT,
    "tools": [read_external_file, internet_search],
    "model": "openai:gpt-4o-mini",  
}

linkedin_subagent = {
    "name": "linked-in-search-agent",
    "description": "Use this agent to look to search linked-in for a company page or its employee profiles.",
    "system_prompt": LI_PROMPT,
    "tools": [read_external_file] + li_tools,
    "model": "openai:gpt-4o-mini",  
}

companies_house_subagent = {
    "name": "companies-house-search-agent",
    "description": "Use this agent to look to search register of incorporated companies.",
    "system_prompt": CH_PROMPT,
    "tools": [read_external_file] + ch_tools,
    "model": "openai:gpt-4o-mini",  
}

### Step 3: Build the main coordinator agent


##### TODOs
 Set_date _ 2015 otherwise most of the searches wont work.

file system and loading of the policy docs
fake and example that means that the companies house tool wont be used.
 

In [110]:
RESEARCH_PROMPT= """

Role: You are a 

Task:
Our plan is to create a comprehensive research report of this user and their loan application
 in which you should check 
Their company's status, the directors
Check for inconstancies across data sources
Is the company's address consistant across  documents?

Check the nature of businss (sometimes called SIC code).
Check the credibility of their  online presence - a company website, staff or directors  that have real people

Highlight any specific concerns.

Ask the user to clarity any concerns or  supply additional evidence.

se the subagents provided to do searches. 

Note that the Companies House subagent should ONLY be used for companies that are incorporated. Other companies such as sole traders
or partnerships are not listed at companies house .

Any infomation discovered should be saved to the file system, and can be used by any of the other sub agents to 
augment their seach.

The context contains the nature of the loan - check /policy_documents and look only in documents that match that loan.

<User details>
{company_context}
</User details>

"""

In [ ]:



application_assessment = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[read_external_file, internet_search] + li_tools+ ch_tools,
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[internet_subagent, linkedin_subagent, companies_house_subagent],
    system_prompt = RESEARCH_PROMPT,
)

In [ ]:
invoke(    
    {"recursion_limit": 20}
)

In [ ]:
# Save application as memory because it will persist across converasations and emails as more information is requested and comes available

# this should be a composite backend.

from deepagents import create_deep_agent
from deepagents.backends import CompositeBackend, StateBackend, StoreBackend
from langgraph.store.memory import InMemoryStore

composite_backend = lambda rt: CompositeBackend(
    default=StateBackend(rt),
    routes={
        "/memories/": StoreBackend(rt),
    }
)

agent = create_deep_agent(
    backend=composite_backend,
    store=InMemoryStore()  # Store passed to create_deep_agent, not backend
)

In [ ]:
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt=""")

### Step 4: Test with A User

In [ ]:
# 1. Input data Rreived from Application process
# Store in LT memory
#Going to put in 2 applications - 1 doctored to see if the research detects what is wrong.

In [ ]:
application1_str = """Applicant Name: Steven Goodman
What loan are you applying for: asset finance
How much would you like to borrow?: £10000
Over what term (# months)?: 24

Is your company registered with Companies House?: Yes

Company Name: Goodmans Consulting

What is your industry?: Business Support Services

When did you start trading?: 11/07/2012

What is your typical annual turnover?: £100000

Last 12 months profit: £84000

Do you expect your income to decrease in the next 12 months?: No

Do you accept card payments?: No

How much is owed to you in invoices?: £0

Monthly expense amount, Not including mortgage or rent repayments: £1500

How much is your monthly rent or mortgage payment?: £3000

How many dependants do you have?: 1

How much do you typically spend on childcare expenses per month?: 200

How much non-business income do you receive each month?: 0

Monthly other household income: 5000


Provide information of all directors or beneficial owners controlling more than 25% of the company:

Person 1:
    First name: Steven
    Surname: Goodman
    Percentage of control: 75%
    Year of Birth: 1972
    Mobile phone number: 012345678
    What is the director's residential status?: Owner With Mortgage
    Residential Address: 3 Manor Road, Ruislip HA47LA"""



In [40]:
#application1_str.split(":|\n")

# import re
# SEPS = (":", "\n")
# rsplit = re.compile("|".join(SEPS)).split
# [s.strip() for s in rsplit(application1_str)]

In [ ]:
# 1. Loan app (input key???) and policy RAG

In [ ]:
### YOUR CODE HERE ###

# Step 1: Define your subagent configurations


# Step 2: Create any additional tools you need


# Step 3: Build the main coordinator agent


# Step 4: Test with a user creating their 30-day challenge


# Step 5: Simulate a daily check-in and adaptation


---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)